In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("/home/shoinoue/Git/CosyVoice/")
sys.path.append("/home/shoinoue/Git/CosyVoice/third_party/Matcha-TTS/")

from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
# import torchaudio
import torch
from scipy.io.wavfile import write as write_wav

import glob
import os
import numpy as np
from tqdm import tqdm

import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))
    
cosyvoice = CosyVoice('/mntcephfs/data/audiow/shoinoue/Model/models/cosyvoice/CosyVoice-300M')

2024-07-31 20:05:56,689 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-07-31 20:05:56,693 - modelscope - INFO - Loading ast index from /home/shoinoue/.cache/modelscope/ast_indexer
2024-07-31 20:05:56,838 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 b628653653bc205329c92249435e1927 and a total number of 980 components indexed


transformer is not installed, please install it if you want to use related modules
failed to import ttsfrd, use WeTextProcessing instead


2024-07-31 20:06:06.915018664 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcurand.so.10: cannot open shared object file: No such file or directory

2024-07-31 20:06:06.915060767 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:743 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
2024-07-31 20:06:07,765 WETEXT INFO found existing fst: /mntcephfs/lab_data/shoinoue/miniconda3/envs/cosyvoice/lib/python3.8/site-packages/tn/zh_tn_tagger.fst
INFO:wetext-zh_normalizer:found existing fst: /mntcephfs/lab_data/shoinoue/miniconda3/envs/cosyvoice/lib/python3.8/site-packages

- Cross Lingual for L2-ARCTIC

In [2]:
# # speakers = ["ASI", "RRBI", "SVBI", "TNI", "ABA", "SKA", "YBAA", "ZHAA"]
# speakers = ["ASI", "RRBI", "SVBI", "TNI"]
# sample_num = 100000000
# repeat_num = 2
# save = False
# play = False

# for spk in speakers:
#     print(spk)
#     base_dir = f"/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/{spk}/wav/"
#     text_dir = "./SPAT/transliteration/"
#     save_dir = f"/mntcephfs/data/audiow/shoinoue/Dataset/CosyVoice/{spk}_allsamples/English/wav/"
#     os.makedirs(save_dir, exist_ok=True)
#     audiofiles = glob.glob(base_dir + "*.wav")
#     audiofiles.sort()
    
#     for path in tqdm(audiofiles[:sample_num]):
#         prompt_speech_16k = load_wav(path, 16000)
#         basename = os.path.basename(path)[:-4]
#         textpath = text_dir + basename + ".npy"
#         sentences = np.load(textpath, allow_pickle=True).item()
#         for r in range(repeat_num):
#             output = cosyvoice.inference_cross_lingual(f'<|en|>{sentences["English"]}', prompt_speech_16k)
#             savepath = save_dir + basename + f"-{r}.wav"
#             if save:
#                 # torchaudio.save(savepath, output['tts_speech'], 22050)
#                 write_wav(savepath, 22050, output["tts_speech"][0].numpy())
#             if play:
#                 x = load_wav(savepath, 16000)
#                 play_audio(prompt_speech_16k, 16000)
#                 play_audio(x, 16000)

- Speaker Accent Different Speech Prompts

In [3]:
def cosyvoice_zeroshot(tts_text, prompt_text, xv_speech_16k, prompt_speech_16k):
    # prompt_speech_16k: speech prompt for accent
    # xv_speech_16k: speech prompt for speaker (xvector)
    
    prompt_text = cosyvoice.frontend.text_normalize(prompt_text, split=False)
    tts_speeches = []
    for i in cosyvoice.frontend.text_normalize(tts_text, split=True):
        model_input = cosyvoice.frontend.frontend_zero_shot(i, prompt_text, xv_speech_16k)
        xv = cosyvoice.frontend.frontend_zero_shot(i, prompt_text, prompt_speech_16k)
        for key in ['llm_prompt_speech_token', 'llm_prompt_speech_token_len', 'llm_embedding']: # llm is speech prompt
        # for key in ['llm_embedding']:
            model_input[key] = xv[key]
        model_output = cosyvoice.model.inference(**model_input)
        tts_speeches.append(model_output['tts_speech'])
    output =  {'tts_speech': torch.concat(tts_speeches, dim=1)}
    return output

In [5]:
# speakers = ["SLT", "BDL", "EEY", "RMS", "AEW", "CLB", "LJM", "LNH"]
speakers = ["SLT", "BDL", "EEY", "RMS"]
text_dir = "./SPAT/transliteration/"
repeat_num = 3

save = True
play = False
for spk in speakers:
    base_dir = f"/mntcephfs/lab_data/shoinoue/Dataset/CMU-ARCTIC/{spk}/wav/"
    prompt_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/ASI/wav/"
    save_dir = f"/mntcephfs/data/audiow/shoinoue/Dataset/CosyVoice/{spk}_allsamples/ASI/wav/"
    os.makedirs(save_dir, exist_ok=True)
    audiofiles = glob.glob(base_dir + "*.wav")
    audiofiles.sort()
    print(spk)
    for p in tqdm(range(len(audiofiles))):
        path = audiofiles[p]
        basename = os.path.basename(path)[:-4]
        start = len(glob.glob(save_dir + basename + f"-*.wav"))
        if start>=repeat_num:
            continue
        xv_speech_16k = load_wav(path, 16000)
        textpath = text_dir + basename + ".npy"
        sentences = np.load(textpath, allow_pickle=True).item()

        # promptpath = prompt_dir + os.path.basename(audiofiles[p+1])[:-4] + ".wav"
        promptpath = prompt_dir + os.path.basename(audiofiles[p])[:-4] + ".wav"
        if not(os.path.exists(promptpath)):
            promptpath = glob.glob(prompt_dir + "*.wav")
            promptpath = promptpath[np.random.randint(len(promptpath))]
        prompt_speech_16k = load_wav(promptpath, 16000)
        prompttextpath = text_dir + os.path.basename(promptpath)[:-4] + ".npy"
        promptsentences = np.load(prompttextpath, allow_pickle=True).item()

        tts_text = f'<|en|>{sentences["English"]}'
        prompt_text = f'<|en|>{promptsentences["English"]}'
        # tts_text = f'{sentences["English"]}'
        # prompt_text = f'{promptsentences["English"]}'
        
        for r in range(start, repeat_num):
            output = cosyvoice_zeroshot(tts_text, prompt_text, xv_speech_16k, prompt_speech_16k)
            savepath = save_dir + basename + f"-{r}.wav"

            if save:
                write_wav(savepath, 22050, output["tts_speech"][0].numpy())
            if play:
                print("accent prompt")
                play_audio(prompt_speech_16k, 16000)
                print("speaker prompt")
                play_audio(xv_speech_16k, 16000)
                if save:
                    print("synthesized")
                    x = load_wav(savepath, 16000)
                    play_audio(x, 16000)

SLT


 28%|██▊       | 312/1132 [13:43<36:03,  2.64s/it]  

KeyboardInterrupt

